In [1]:
# Climatalogical Rain and PET per month for CV and among year changes

import arcpy
import csv
DaysRain = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
DaysMonth = [31, 28, 31, 30, 31, 30, 31,31,30,31,30,31]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

FolderBase = r"C:\\DannyData\\Projects\\Agrichange\\Climate data\\"
FolderOut = FolderBase +"TemporaryOutputs\\"
FolderRain = FolderBase + "GPCC rain\\"
FolderPET = FolderBase + "PET CRU\\"

NCInfileA0 = FolderRain + "full_data_monthly_v2022_1951_1960_025.nc"
NCInfileA1 = FolderRain + "full_data_monthly_v2022_1961_1970_025.nc"
NCInfileA2 = FolderRain + "full_data_monthly_v2022_1971_1980_025.nc"
NCInfileA3 = FolderRain + "full_data_monthly_v2022_1981_1990_025.nc"
NCInfileA4 = FolderRain + "full_data_monthly_v2022_1991_2000_025.nc"
NCInfileA5 = FolderRain + "full_data_monthly_v2022_2001_2010_025.nc"
NCInfileA6 = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"
NCInfileA7 = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"

NCInfileB0 = FolderPET + "cru_ts4.07.1951.1960.pet.dat.nc" # Note all per day!!
NCInfileB1 = FolderPET + "cru_ts4.07.1961.1970.pet.dat.nc"
NCInfileB2 = FolderPET + "cru_ts4.07.1971.1980.pet.dat.nc"
NCInfileB3 = FolderPET + "cru_ts4.07.1981.1990.pet.dat.nc"
NCInfileB4 = FolderPET + "cru_ts4.07.1991.2000.pet.dat.nc"
NCInfileB5 = FolderPET + "cru_ts4.07.2001.2010.pet.dat.nc"
NCInfileB6 = FolderPET + "cru_ts4.07.2011.2020.pet.dat.nc"
NCInfileB7 = FolderPET + "cru_ts4.07.2021.2022.pet.dat.nc"


VariableTypeRain = "precip"
VariableTypePET = "pet"

InputLocations = "LTE-FullSet"
Start_Year = 2011
End_YearEntered = 2020
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

OutFileRain = FolderOut + "CVRainFall" + ".csv"
OutFilePET = FolderOut + "CVPET" + ".csv"
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        # Extract and Name Days per year
        if year < 1961:
            NCInfileRain = NCInfileA0
            NCInfilePET = NCInfileB0
        elif 1961 <= year <= 1970:    
            NCInfileRain = NCInfileA1
            NCInfilePET = NCInfileB1
        elif 1971 <= year <= 1980:
            NCInfileRain = NCInfileA2
            NCInfilePET = NCInfileB2
        elif 1981 <= year <= 1990:
            NCInfileRain = NCInfileA3
            NCInfilePET = NCInfileB3
        elif 1991 <= year <= 2000:
            NCInfileRain = NCInfileA4
            NCInfilePET = NCInfileB4
        elif 2001 <= year <= 2010:
            NCInfileRain = NCInfileA5
            NCInfilePET = NCInfileB5
        elif 2011 <= year <= 2020:
            NCInfileRain = NCInfileA6
            NCInfilePET = NCInfileB6
        else:
            NCInfileRain = NCInfileA7
            NCInfilePET = NCInfileB7

        Count = 0
        for date in range(0,len(DaysRain)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            
            Count = Count + 1;
            DimensionRain = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNameRain = "RainDates_" + str(Count)
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNamePET = "PETDates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileRain, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNameRain, band_dimension="", dimension_values=DimensionRain,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            if date == 0:
                LayersArrayFullRain = LayerNameRain
            else:
                LayersArrayFullRain = LayersArrayFullRain + ";" + LayerNameRain   
            
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=DaysMonth[date])
            out_raster.save(LayerNamePET)
            
            if date == 0:
                LayersArrayFullPET = LayerNamePET
            else:
                LayersArrayFullPET = LayersArrayFullPET + ";" + LayerNamePET    
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', extent='-192.193863381631 -148.729622931763 324.622704190431 131.455603150768 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
            OutMapRain = FolderOut + "Precipitation_" + "Y_" + str(year) +".tif"       
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRain,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
            out_raster.save("MEANFull")
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRain,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT") 
            out_raster.save(OutMapRain)
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRain,
                     statistics_type="STD",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
            out_raster.save("STDFull") 
            OutMapRain = FolderOut + "CVPrecipitation_" + "Y_" + str(year) +".tif"  
            out_raster = arcpy.sa.Divide(in_raster_or_constant1="STDFull",in_raster_or_constant2= "MEANFull")
            out_raster.save("RainFallYearFile")
            out_raster.save(OutMapRain)
        
        OutMapPET = FolderOut + "PET_" + "Y_" + str(year) +".tif" 
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("MEANFull")  
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save(OutMapPET)
        
        OutMapRain = FolderOut + "CVPET_" + "Y_" + str(year) +".tif" 
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="STD",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("STDFull")   
        out_raster = arcpy.sa.Divide(in_raster_or_constant1="STDFull",in_raster_or_constant2= "MEANFull")
        out_raster.save("PETFallYearFile")     
        out_raster.save(OutMapRain)
        
        TableNameFullRain = "ZonalRain"
        TableNameFullPET = "ZonalPET"
                        
        if  year == Start_Year:
            YeartmpRain = TableNameFullRain
            YeartmpPET = TableNameFullPET
        else:
            YeartmpRain = "TmpYearRain"
            YeartmpPET = "TmpYearPET"
        YearName = "Y_" + str(year)        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "RainFallYearFile", out_table=YeartmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpRain,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "PETFallYearFile", out_table=YeartmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpPET,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableNameFullRain,in_field="Locations",join_table=YeartmpRain,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
            arcpy.management.JoinField(in_data=TableNameFullPET,in_field="Locations",join_table=YeartmpPET,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
        
        #Write away per year 
        for i in range(0,2):
            if i == 0:
                table = TableNameFullRain 
                OutFile = OutFileRain
            elif i == 1:
                table = TableNameFullPET
                OutFile = OutFilePET  
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)    
print("Finished                              ")        

Finished                              


In [2]:
# TEMPERATURES 

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDay = ["Jan5", "Jan15", "Jan25", "Feb5", "Feb15", "Feb25", "March5", "March15", "March25", "Apr5", "Apr15", "Apr25", \
           "May5", "May15", "May25", "June5", "June15", "June25", "July5", "July15", "July25", "Aug5", "Aug15", "Aug25", \
           "Sept5", "Sept15", "Sept25", "Oct5", "Oct15", "Oct25", "Nov5", "Nov15", "Nov25", "Dec5", "Dec15", "Dec25"]
FolderBase = r"C:\\DannyData\\Projects\\Agrichange\\Climate data\\Agriclimatic indicators Copernicus\\"
#FolderGrowthSeason = FolderBase + "Mean of daily man temperature\\"
FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"


Folder = FolderBase + "Mean of daily mean temperature\\"
NCInfile1 = Folder + "TG_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
NCInfile2 = Folder + "TG_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
NCInfile3 = Folder + "TG_C3S-glob-agric_hadgem2-es_rcp8p5_dek_20110101-20401231_v1.1.nc"
TypeStats = "MEAN"
NameIndicator = "MEAN"
VariableType = "TG"

InputLocations = "LTE-FullSet"
Start_Year = 1968
End_Year = 1970 # note add 1
YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileYear = FolderOut + NameIndicator + "Temp_PerYear"  + ".csv"
        OutFileMonth = FolderOut + NameIndicator + "Temp_PerMonth"  + ".csv"
        # Extract and Name Days per year
        if year < 1981:
            NCInfile = NCInfile1
        elif year > 2010:
            NCInfile = NCInfile3
        else:
            NCInfile = NCInfile2
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            YearMonthCount = YearMonthCount + 1
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmp = "TmpDates_" + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer=LayerNametmp, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmp,in_raster_or_constant2= 273.15)
            out_raster.save(LayerName) 
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
                
            TableName = "ZonalTemp"
            if  YearMonthCount == 1:
                TableNametmp = TableName
            else:
                TableNametmp = "TmpTable"           
            
            NewName = "Y_" + str(year) +"_" + MonthDay[date]
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= LayerName, out_table=TableNametmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmp,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            if YearMonthCount != 1:
                arcpy.management.JoinField(in_data=TableName,in_field="Locations",join_table=TableNametmp,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)            
        OutMapTemp = FolderOut + "MainDailyTemp_" + "Y_" + str(year) +".tif"       
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRast") 
        out_raster.save(OutMapTemp) 
        
        TableYear = "YearTemp"
        if  year == Start_Year:
            Yeartmp = TableYear
        else:
            Yeartmp = "TmpYear"
        YearName = "Y_" + str(year)    
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRast", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYear,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        #end of per LTE loop 
        #Write away per year 
        for i in range(0,2):
            if i == 0:
                table = TableName
                OutFile = OutFileMonth
            else:
                table = TableYear
                OutFile = OutFileYear
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)    
print("Finished                              ")                

Finished                              


In [1]:
# TEMPERATURE RANGE 

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"

FolderA = "C:\DannyData\Projects\Agrichange\Climate data\Agriclimatic indicators Copernicus\Maximum value of daily maximum temperature\\"
FolderB = "C:\DannyData\Projects\Agrichange\Climate data\Agriclimatic indicators Copernicus\Minimum value of the daily minimum Temperature\\"

NCInfileA1 = FolderA + "TXx_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
NCInfileA2 = FolderA + "TXx_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
NCInfileA3 = FolderA + "TXx_C3S-glob-agric_hadgem2-es_rcp8p5_dek_20110101-20401231_v1.1.nc"
NCInfileB1 = FolderB + "TNn_C3S-glob-agric_hadgem2-es_hist_dek_19510101-19801231_v1.1.nc"
NCInfileB2 = FolderB + "TNn_C3S-glob-agric_hadgem2-es_hist_dek_19810101-20101231_v1.1.nc"
NCInfileB3 = FolderB + "TNn_C3S-glob-agric_hadgem2-es_rcp8p5_dek_20110101-20401231_v1.1.nc"
TypeStats = "MEAN"
VariableTypeA = "TXx"
VariableTypeB = "TNn"

InputLocations = "LTE-FullSet"
Start_Year = 2020
End_Year = 2021 #note add 1

OutFileDaily = FolderOut + "DailyTempRange" + ".csv"
OutFileMax = FolderOut + "YearMaxTemp" + ".csv"
OutFileMin = FolderOut + "YearMinTemp" + ".csv"
OutFileAnnualRange = FolderOut + "AnnualTempRange" + ".csv"
OutFileIso = FolderOut + "Isothermality" + ".csv"

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        # Extract and Name Days per year
        if year < 1981:
            NCInfileA = NCInfileA1
            NCInfileB = NCInfileB1
        elif year > 2010:
            NCInfileA = NCInfileA3
            NCInfileB = NCInfileB3
        else:
            NCInfileA = NCInfileA2
            NCInfileB = NCInfileB2
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpA = "TmpDatesA_" + str(Count)
            LayerNametmpB = "TmpDatesB_" + str(Count)
            LayerNameDaily = "RangeDates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileA, variable=VariableTypeA, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpA, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileB, variable=VariableTypeB, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpB, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmpA,in_raster_or_constant2= LayerNametmpB)
            out_raster.save( LayerNameDaily) 
            # add here per month per LTE and saving

            if date == 0:
                LayersArrayFullDaily =  LayerNameDaily
                LayersArrayFullMaxYear = LayerNametmpA
                LayersArrayFullMinYear = LayerNametmpB
            else:
                LayersArrayFullDaily = LayersArrayFullDaily + ";" +  LayerNameDaily   
                LayersArrayFullMaxYear = LayersArrayFullMaxYear + ";" +  LayerNametmpA
                LayersArrayFullMinYear = LayersArrayFullMinYear + ";" +  LayerNametmpB
                
        YearName = "Y_" + str(year)        
        #Daily Range
        OutMapDaily = FolderOut + "DailyTempRange_" + "Y_" + str(year) +".tif" 
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullDaily,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("DailyRange") 
        out_raster.save(OutMapDaily ) 
        
        TableYearDaily = "ZonalFullDaily"
        if  year == Start_Year:
            Yeartmp = TableYearDaily
        else:
            Yeartmp = "TmpYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "DailyRange", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYearDaily,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        
        # Maximum
        OutMapMax = FolderOut + "MaxTemp_" + "Y_" + str(year) +".tif" 
        TmpMax = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullMaxYear,
                     statistics_type="MAXIMUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster = arcpy.sa.Minus(in_raster_or_constant1=TmpMax,in_raster_or_constant2= 273.15)
        out_raster.save("MaximumTemp") 
        out_raster.save(OutMapMax) 
        
        TableMaxYear = "ZonalMaximum"
        if  year == Start_Year:
            YearMaxtmp = TableMaxYear
        else:
            YearMaxtmp = "TmpMaxYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "MaximumTemp", out_table=YearMaxtmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearMaxtmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableMaxYear,in_field="Locations",join_table=YearMaxtmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
       
        # Minimum
        OutMapMin = FolderOut + "MinTemp_" + "Y_" + str(year) +".tif" 
        TmpMin = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullMinYear,
                     statistics_type="MINIMUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster = arcpy.sa.Minus(in_raster_or_constant1=TmpMin,in_raster_or_constant2= 273.15)
        out_raster.save("MinimumTemp") 
        out_raster.save(OutMapMin) 
        
        TableMinYear = "ZonalMinimum"
        if  year == Start_Year:
            YearMintmp = TableMinYear
        else:
            YearMintmp = "TmpMinYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "MinimumTemp", out_table=YearMintmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearMintmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableMinYear,in_field="Locations",join_table=YearMintmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)      
        
        # Annual Range
        OutMapAnnualRange = FolderOut + "AnnualTempRange_" + "Y_" + str(year)  +".tif"  
        out_raster = arcpy.sa.Minus(in_raster_or_constant1="MaximumTemp",in_raster_or_constant2= "MinimumTemp")
        out_raster.save("AnnualRange")
        out_raster.save(OutMapAnnualRange)
        TableRangeYear = "ZonalRange"
        if  year == Start_Year:
            YearRangetmp = TableRangeYear
        else:
            YearRangetmp = "TmpRangeYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "AnnualRange", out_table=YearRangetmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearRangetmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableRangeYear,in_field="Locations",join_table=YearRangetmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)      
        
        # Isothermality
        OutMapIso = FolderOut + "Isothermality_" + "Y_" + str(year) +".tif" 
        IsoTemp = arcpy.sa.Divide(in_raster_or_constant1="DailyRange",in_raster_or_constant2= "AnnualRange")
        out_raster = arcpy.sa.Times(in_raster_or_constant1=IsoTemp,in_raster_or_constant2= 100)
        out_raster.save("Isothermality") 
        out_raster.save(OutMapIso)
        TableIsoYear = "ZonalIso"
        if  year == Start_Year:
            YearIsotmp = TableIsoYear
        else:
            YearIsotmp = "TmpIsoYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "Isothermality", out_table=YearIsotmp ,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearIsotmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableIsoYear,in_field="Locations",join_table=YearIsotmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)     
        
        #end of per LTE loop 
        #Write away per year 
        for i in range(0,5):
            if i == 0:
                table = TableYearDaily
                OutFile = OutFileDaily
            elif i == 1:
                table = TableMaxYear
                OutFile = OutFileMax
            elif i == 2:    
                table = TableMinYear
                OutFile = OutFileMin
            elif i == 3:    
                table = TableRangeYear
                OutFile = OutFileAnnualRange
            elif i == 4:    
                table = TableIsoYear
                OutFile = OutFileIso
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals) 
        arcpy.Delete_management(LayersArrayFullDaily)
        arcpy.Delete_management(LayersArrayFullMaxYear)
        arcpy.Delete_management(LayersArrayFullMinYear )    
print("Finished                              ")    

Finished                              


In [1]:
# Climatalogical water deficit with start at 1-01 every year

import arcpy
import csv
DaysRain = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
DaysMonth = [31, 28, 31, 30, 31, 30, 31,31,30,31,30,31]

FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"

NCInfileA1 = r"C:\DannyData\Projects\Agrichange\Climate data\GPCC rain\full_data_monthly_v2022_1961_1970_025.nc"
NCInfileA2 = r"C:\DannyData\Projects\Agrichange\Climate data\GPCC rain\full_data_monthly_v2022_1971_1980_025.nc"
NCInfileA3 = r"C:\DannyData\Projects\Agrichange\Climate data\GPCC rain\full_data_monthly_v2022_1981_1990_025.nc"
NCInfileA4 = r"C:\DannyData\Projects\Agrichange\Climate data\GPCC rain\full_data_monthly_v2022_1991_2000_025.nc"
NCInfileA5 = r"C:\DannyData\Projects\Agrichange\Climate data\GPCC rain\full_data_monthly_v2022_2001_2010_025.nc"
NCInfileA6 = r"C:\DannyData\Projects\Agrichange\Climate data\GPCC rain\full_data_monthly_v2022_2011_2020_025.nc"

NCInfileB1 = r"C:\DannyData\Projects\Agrichange\Climate data\PET CRU\cru_ts4.07.1961.1970.pet.dat.nc"
NCInfileB2 = r"C:\DannyData\Projects\Agrichange\Climate data\PET CRU\cru_ts4.07.1971.1980.pet.dat.nc"
NCInfileB3 = r"C:\DannyData\Projects\Agrichange\Climate data\PET CRU\cru_ts4.07.1981.1990.pet.dat.nc"
NCInfileB4 = r"C:\DannyData\Projects\Agrichange\Climate data\PET CRU\cru_ts4.07.1991.2000.pet.dat.nc"
NCInfileB5 = r"C:\DannyData\Projects\Agrichange\Climate data\PET CRU\cru_ts4.07.2001.2010.pet.dat.nc"
NCInfileB6 = r"C:\DannyData\Projects\Agrichange\Climate data\PET CRU\cru_ts4.07.2011.2020.pet.dat.nc"


TypeStats = "MAXIMUM"
NameIndicator = "CWDAnnual"
VariableTypeRain = "precip"
VariableTypePET = "pet"

InputLocations = "LTE-FullSet"
Start_Year = 1974
End_YearEntered = 2020 
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)
Previous_Month = 0 
Start_Seq = 0

YearCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        YearCount = YearCount + 1
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileFull = FolderOut + NameIndicator + ".csv"
        # Extract and Name Days per year
        if year < 1971:
            NCInfileRain = NCInfileA1
            NCInfilePET = NCInfileB1
        elif 1971 <= year <= 1980:
            NCInfileRain = NCInfileA2
            NCInfilePET = NCInfileB2
        elif 1981 <= year <= 1990:
            NCInfileRain = NCInfileA3
            NCInfilePET = NCInfileB3
        elif 1991 <= year <= 2000:
            NCInfileRain = NCInfileA4
            NCInfilePET = NCInfileB4
        elif 2001 <= year <= 2010:
            NCInfileRain = NCInfileA5
            NCInfilePET = NCInfileB5
        else:
            NCInfileRain = NCInfileA6
            NCInfilePET = NCInfileB6
        # Water deficit per calender year, so every year a new one with zeros is generated    
        arcpy.Delete_management("Previous_MonthDeficit") 
        out_raster = arcpy.sa.CreateConstantRaster(constant_value=0,data_type="INTEGER",cell_size=0.25,
                   extent='-180 -90 180 90 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]')
        out_raster.save("Previous_MonthDeficit")
        
        Count = 0
        for date in range(0,len(DaysRain)):
            Count = Count + 1;
            DimensionRain = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpRain = "TmpDatesRain_" + str(Count)
            LayerNametmpPET = "TmpDatesPET_"  + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.Delete_management(LayerNametmpRain) 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileRain, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpRain, band_dimension="", dimension_values=DimensionRain,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=DaysMonth[date])
            out_raster.save(LayerNametmpPET)
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmpPET,in_raster_or_constant2= LayerNametmpRain)
            out_raster.save("InBetweenCMW")
            
            out_raster = arcpy.sa.Plus(in_raster_or_constant1="InBetweenCMW",in_raster_or_constant2= "Previous_MonthDeficit")
            out_raster.save("InBetweenCMW2")
            arcpy.Delete_management("Previous_MonthDeficit") 
            out_raster = arcpy.sa.Con(in_conditional_raster="InBetweenCMW2",in_true_raster_or_constant=0,
                         in_false_raster_or_constant="InBetweenCMW2",where_clause="VALUE < 0")
            out_raster.save(LayerName)
            
            arcpy.management.CopyRaster(in_raster=LayerName,out_rasterdataset="Previous_MonthDeficit",config_keyword="",background_value=None,
                                        nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                                        RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                                        build_multidimensional_transpose="NO_TRANSPOSE")            
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        OutMapCWD = FolderOut + "CWDTemp_" + "Y_" + str(year) +".tif" 
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("CWDSumFull")
        if year > 2010:
            out_raster.save(OutMapCWD)
        TableNameFull = "ZonalCWD"
        if  YearCount == 1:
            TableNametmp = TableNameFull
        else:
            TableNametmp = "TmpSeasonYear"
        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "CWDSumFull", out_table=TableNametmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        NewName = "Y_" + str(year)
        arcpy.management.AlterField(in_table=TableNametmp,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if YearCount != 1:
            arcpy.management.JoinField(in_data=TableNameFull,in_field="Locations",join_table=TableNametmp,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)   

        #end of per LTE loop 
        table = TableNameFull 
        OutFile = OutFileFull
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFile,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)        

In [3]:
# For multi year averages for Rainfall changes for figures

import arcpy
import csv
DaysRain = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

FolderBase = r"C:\\DannyData\\Projects\\Agrichange\\Climate data\\"
FolderOutPut = FolderBase +"TemporaryOutputs\\"
FolderRain = FolderBase + "GPCC rain\\"

NCInfileRain = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"
VariableTypeRain = "precip"
InputLocations = "LTE-FullSet"
Start_Year = 2011
End_YearEntered = 2020
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

CountYear = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        # Extract and Name Days per year
        CountYear = CountYear +1 
        Count = 0
        for date in range(0,len(DaysRain)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            Count = Count + 1; 
            DimensionRain = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNameRain = "RainDates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileRain, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNameRain, band_dimension="", dimension_values=DimensionRain,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            if date == 0:
                LayersArrayFullRain = LayerNameRain
            else:
                LayersArrayFullRain = LayersArrayFullRain + ";" + LayerNameRain 
        YearName = "RainFallY_" + str(year)              
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster=None, extent='-217.222823048928 -113.943744937245 242.018604784816 135.027885149131 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask=None, scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRain,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")  
            out_raster.save(YearName)     
            if CountYear == 1:
                LayersArrayYears =YearName
            else:
                LayersArrayYears = LayersArrayYears + ";" + YearName 
    with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster=None, extent='-217.222823048928 -113.943744937245 242.018604784816 135.027885149131 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask=None, scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayYears,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")   
        out_raster.save(r"C:\DannyData\Projects\Agrichange\Climate data\RainFall11_20.tif")
print("Finished                              ")        

Finished                              


In [9]:
# For multi year averages for Temperature changes for figures
# TEMPERATURES 

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDay = ["Jan5", "Jan15", "Jan25", "Feb5", "Feb15", "Feb25", "March5", "March15", "March25", "Apr5", "Apr15", "Apr25", \
           "May5", "May15", "May25", "June5", "June15", "June25", "July5", "July15", "July25", "Aug5", "Aug15", "Aug25", \
           "Sept5", "Sept15", "Sept25", "Oct5", "Oct15", "Oct25", "Nov5", "Nov15", "Nov25", "Dec5", "Dec15", "Dec25"]
Folder = r"C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\RCP6_0HADGEM2 projections Temprature Copernicus\\"
NCInfile = Folder + "TNn_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
TypeStats = "MEAN"
NameIndicator = "MEAN"
VariableType = "TNn"

InputLocations = "LTE-FullSet"
Start_Year = 2090
End_Year = 2100 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    CountYear = 0
    for year in range(Start_Year,End_Year):
        CountYear = CountYear + 1;
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmp = "TmpDates_" + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer=LayerNametmp, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmp,in_raster_or_constant2= 273.15)
            out_raster.save(LayerName) 
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName        
        YearName = "RainFallY_" + str(year)
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster=None, extent='-217.222823048928 -113.943744937245 242.018604784816 135.027885149131 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask=None, scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
            out_raster.save(YearName) 
        if CountYear == 1:
            LayersArrayYears =YearName
        else:
            LayersArrayYears = LayersArrayYears + ";" + YearName 
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster=None, extent='-217.222823048928 -113.943744937245 242.018604784816 135.027885149131 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask=None, scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayYears,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")   
            out_raster.save(r"C:\DannyData\Projects\Agrichange\Climate data\MinTemp90_99.tif")              
print("Finished                              ")                

Finished                              


In [6]:
# Extract Precipiation from Imerg for 2019 to 2022 period, because not on GPCC
# Sum to monthly 

import arcpy
import csv

Months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
Days = ["01","02","03","04","05","06","07","08","09","10","11","12","13",\
        "14","15","16","17","18","19","20","21","22","23","24","25","26",\
        "27","28","29","30","31"]
DaysperMonth = [31,28,31,30,31,30,31,31,30,31,30,31]
Years = ["2019","2020","2021","2022"]
MonthstoRun = [12,12,12,12]

InputLocations = "LTE-FullSet"
YearMonthCount = 0
OutFileRainSummation = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\ImMERGRain.csv"  
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(0,len(Years)):
        EndMonth = MonthstoRun[year]
        for month in range(0,EndMonth):
            EndDay = DaysperMonth[month]
            YearMonthCount = YearMonthCount + 1
            OutFileMonth = "Month_" + Years[year] + Months[month]  
            Message = "Running Month #"+ Months[month] + " in year " + Years[year] + " with " + str(EndDay) + " days           "
            print(Message, end="\r") 
            DayCount = 0
            for day in range (0,EndDay):
                InFileNC = "C:\\DannyData\\Projects\\Adapts\\Data\\Imerg 2021-2023(Rainfall per day)\\" + Years[year] +"\\3B-DAY-L.MS.MRG.3IMERG." +\
                            Years[year] + Months[month] + Days[day] + "-S000000-E235959.V06.nc4"   
                OutFileRain = "IMERG_day_" + Months(month) + '_' + Days[day]
                OutFilePresence = "Presence_" + Days[day]
                arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=InFileNC,variable="HQprecipitation",
                         x_dimension="lon",y_dimension="lat",out_raster_layer="Precipitation_LayerDay",
                        band_dimension="time",dimension_values=None,value_selection_method="BY_VALUE",cell_registration="CENTER")
                out_raster = arcpy.sa.Times("Precipitation_LayerDay", 1)
                out_raster.save(OutFileRain)
            
                if DayCount == 0:
                    LayersArrayFull = OutFileRain
                else:
                    LayersArrayFull = LayersArrayFull + ";" +  OutFileRain
                DayCount = Count = 1;   
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull ,
                           statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                           percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
            out_raster.save(OutFileMonth)
            arcpy.Delete_management(LayersArrayFull)   
            
            TableNameFullRain = "ZonalRain"
            if  YearMonthCount == 1:
                TableNametmpRain = TableNameFullRain
            else:
                TableNametmpRain = "TmpRain"
            NewName = "Y_" + str(year) +"_" + Months[month]
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= OutFileMonth, out_table=TableNametmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmpRain,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            if  YearMonthCount != 1:
                arcpy.management.JoinField(in_data=TableNameFullRain,in_field="Locations",join_table=TableNametmpRain,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)    
            
            #Write away per Month 
            table = TableNameFullRain 
            OutFile = OutFileRainSummation
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)
        
print("Finished                              ")                     

Finished                               days           


In [8]:
# Climatalogical Rain and PET per month for CWD and among year changes 

import arcpy
import csv
DaysRain = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
DaysMonth = [31, 28, 31, 30, 31, 30, 31,31,30,31,30,31]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

FolderBase = r"C:\DannyData\Projects\Agrichange\Climate data\\"
#FolderGrowthSeason = FolderBase + "Agriclimatic indicators Copernicus\\Mean of daily mean temperature\\"
FolderOutPut = FolderBase +"TemporaryOutputs\\"
FolderRain = FolderBase + "GPCC rain\\"
FolderPET = FolderBase + "PET CRU\\"

NCInfileA0 = FolderRain + "full_data_monthly_v2022_1951_1960_025.nc"
NCInfileA1 = FolderRain + "full_data_monthly_v2022_1961_1970_025.nc"
NCInfileA2 = FolderRain + "full_data_monthly_v2022_1971_1980_025.nc"
NCInfileA3 = FolderRain + "full_data_monthly_v2022_1981_1990_025.nc"
NCInfileA4 = FolderRain + "full_data_monthly_v2022_1991_2000_025.nc"
NCInfileA5 = FolderRain + "full_data_monthly_v2022_2001_2010_025.nc"
NCInfileA6 = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"
NCInfileA7 = FolderRain + "full_data_monthly_v2022_2011_2020_025.nc"

NCInfileB0 = FolderPET + "cru_ts4.07.1951.1960.pet.dat.nc"
NCInfileB1 = FolderPET + "cru_ts4.07.1961.1970.pet.dat.nc"
NCInfileB2 = FolderPET + "cru_ts4.07.1971.1980.pet.dat.nc"
NCInfileB3 = FolderPET + "cru_ts4.07.1981.1990.pet.dat.nc"
NCInfileB4 = FolderPET + "cru_ts4.07.1991.2000.pet.dat.nc"
NCInfileB5 = FolderPET + "cru_ts4.07.2001.2010.pet.dat.nc"
NCInfileB6 = FolderPET + "cru_ts4.07.2011.2020.pet.dat.nc"
NCInfileB7 = FolderPET + "cru_ts4.07.2021.2022.pet.dat.nc"


VariableTypeRain = "precip"
VariableTypePET = "pet"

InputLocations = "LTE-FullSet"
Start_Year = 1955
End_YearEntered = 2022
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        OutFileRain = FolderOutPut + "MonthlyRainFall" + ".csv"
        OutFilePET = FolderOutPut + "MonthlyPET" + ".csv"
        OutFileRainYear = FolderOutPut + "AnnualRainFall" + ".csv"
        OutFilePETYear = FolderOutPut + "AnnualPET" + ".csv"
        # Extract and Name Days per year
        if year < 1961:
            NCInfileRain = NCInfileA0
            NCInfilePET = NCInfileB0
        elif 1961 <= year <= 1970:    
            NCInfileRain = NCInfileA1
            NCInfilePET = NCInfileB1
        elif 1971 <= year <= 1980:
            NCInfileRain = NCInfileA2
            NCInfilePET = NCInfileB2
        elif 1981 <= year <= 1990:
            NCInfileRain = NCInfileA3
            NCInfilePET = NCInfileB3
        elif 1991 <= year <= 2000:
            NCInfileRain = NCInfileA4
            NCInfilePET = NCInfileB4
        elif 2001 <= year <= 2010:
            NCInfileRain = NCInfileA5
            NCInfilePET = NCInfileB5
        elif 2011 <= year <= 2020:
            NCInfileRain = NCInfileA6
            NCInfilePET = NCInfileB6
        else:
            NCInfileRain = NCInfileA7
            NCInfilePET = NCInfileB7
            
        RainSumLayer = "RainSum"
        PETSumLayer = "SumPET"    
        Count = 0
        for date in range(0,len(DaysRain)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            Count = Count + 1;
            YearMonthCount = YearMonthCount + 1
            DimensionRain = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpRain = "TmpDatesRain"
            LayerNametmpPET = "TmpDatesPET"
            arcpy.Delete_management(LayerNametmpRain) 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileRain, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenRain", band_dimension="", dimension_values=DimensionRain,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenRain",in_raster_or_constant2=1)
            out_raster.save(LayerNametmpRain)
            
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=DaysMonth[date])
            out_raster.save(LayerNametmpPET)
            
            if Count == 1:
                arcpy.Delete_management(RainSumLayer)
                arcpy.management.CopyRaster(in_raster=LayerNametmpRain,out_rasterdataset=RainSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")
                arcpy.Delete_management(PETSumLayer)
                arcpy.management.CopyRaster(in_raster=LayerNametmpPET,out_rasterdataset=PETSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
            else:
                out_raster = arcpy.sa.Plus(in_raster_or_constant1=RainSumLayer,in_raster_or_constant2= LayerNametmpRain)
                out_raster.save("InBetweenRainSum")
                arcpy.Delete_management(RainSumLayer)
                arcpy.management.CopyRaster(in_raster="InBetweenRainSum",out_rasterdataset=RainSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
                arcpy.Delete_management("InBetweenRainSum")
                
                out_raster = arcpy.sa.Plus(in_raster_or_constant1=PETSumLayer,in_raster_or_constant2= LayerNametmpPET)
                out_raster.save("InBetweenPETSum")
                arcpy.Delete_management(PETSumLayer)
                arcpy.management.CopyRaster(in_raster="InBetweenPETSum",out_rasterdataset=PETSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
                arcpy.Delete_management("InBetweenPETSum")
            
            TableNameFullRain = "ZonalRain"
            TableNameFullPET = "ZonalPET"
            if  YearMonthCount == 1:
                TableNametmpRain = TableNameFullRain
                TableNametmpPET = TableNameFullPET
            else:
                TableNametmpRain = "TmpRain"
                TableNametmpPET = "TmpPET"
            
            NewName = "Y_" + str(year) +"_" + Months[date]
            
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= LayerNametmpRain, out_table=TableNametmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmpRain,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= LayerNametmpPET, out_table=TableNametmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmpPET,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            if  YearMonthCount != 1:
                arcpy.management.JoinField(in_data=TableNameFullRain,in_field="Locations",join_table=TableNametmpRain,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)    
                arcpy.management.JoinField(in_data=TableNameFullPET,in_field="Locations",join_table=TableNametmpPET,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
                
        TableYearRain = "YearRain"
        TableYearPET = "YearPET"
        if  year == Start_Year:
            YeartmpRain = TableYearRain
            YeartmpPET = TableYearPET
        else:
            YeartmpRain = "TmpYearRain"
            YeartmpPET = "TmpYearPET"
        YearName = "Y_" + str(year)        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= RainSumLayer, out_table=YeartmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpRain,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= PETSumLayer, out_table=YeartmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpPET,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYearRain,in_field="Locations",join_table=YeartmpRain,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
            arcpy.management.JoinField(in_data=TableYearPET,in_field="Locations",join_table=YeartmpPET,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
        
         #Write away per year 
        for i in range(0,4):
            if i == 0:
                table = TableNameFullRain 
                OutFile = OutFileRain
            elif i == 1:
                table = TableNameFullPET
                OutFile = OutFilePET  
            elif i == 2:
                table = TableYearRain
                OutFile = OutFileRainYear 
            else:
                table = TableYearPET
                OutFile = OutFilePETYear
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)    
print("Finished                              ")                    

Finished                              


In [4]:
# Radiation 

import arcpy
import csv
#Days = ["16/01", "14/02", "16/03", "15/04", "16/05", "15/06", "16/07", "16/08", "15/09", "16/10", "15/11", "16/12"] #Europe
Days = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"] #Africa

#NCInfile = r"C:\DannyData\Projects\Agrichange\Climate data\RadiationKNMI\Europe\qq_0.25deg_reg_v28.0eu_mo.nc" #Europe
NCInfile = r"C:\DannyData\Projects\Agrichange\Climate data\RadiationKNMI\Africa\sis_cmsaf.nc" #Africa
TypeStats = "MEAN"
#VariableType = "qq" #Europe
VariableType = "SIS" #Europe
InputLocations = "LTE-FullSet"
Start_Year = 1983
End_Year = 2023 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileYear = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\MeanRadation_PerYear.csv" #Mean
        #OutFileYear = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\MaxRadation_PerYear.csv" #Maximum
        # Extract and Name Days per year
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
           # if date == 1: # For Europe Only
           #     if year/4 == round(year/4):
           #         print("'Leap Year                         ")
           #         Dimension = "time '"+ "15/02" +"/" +str(year) +"  12:00:00 AM'" 
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer="LayerNameTmp", band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times("LayerNameTmp", 1)
            out_raster.save(LayerName)
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRast") 
        arcpy.Delete_management(LayersArrayFull)   
        TableYear = "YearTemp"
        if  year == Start_Year:
            Yeartmp = TableYear
        else:
            Yeartmp = "TmpYear"
        YearName = "Y_" + str(year)    
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRast", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYear,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        #end of per LTE loop 
        #Write away per year 
        table = TableYear
        OutFile = OutFileYear
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFile,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)    
print("Finished                                                                   ")

Finished                                                                   


In [1]:
# OZone

import arcpy
import csv
Days = ["01/01", "01/02", "01/03", "01/04", "01/05", "01/06", "01/07", "01/08", "01/09", "01/10", "01/11", "01/12"]
NCInfile = r"C:\DannyData\Projects\Agrichange\Climate data\OzoneLayer79_23(KNMI)\o3col1.nc"
TypeStats = "MEAN"
VariableType = "Average_O3_column"
InputLocations = "LTE-FullSet"
Start_Year = 1979
End_Year = 2023 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileYear = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\MeanOzone_PerYear.csv"
        # Extract and Name Days per year
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="longitude", y_dimension="latitude",
            out_raster_layer="LayerNameTmp", band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times("LayerNameTmp", 1)
            out_raster.save(LayerName)
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRast") 
        
        TableYear = "YearTemp"
        if  year == Start_Year:
            Yeartmp = TableYear
        else:
            Yeartmp = "TmpYear"
        YearName = "Y_" + str(year)    
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRast", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYear,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        #end of per LTE loop 
        #Write away per year 
        table = TableYear
        OutFile = OutFileYear
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFile,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)    
print("Finished                                      ")

Finished                                      


In [1]:
# population pressure as function of distance

import arcpy
import csv

Years = [1975,1980,1985,1990,1995,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,\
         2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,20001,20051,20101,20151,20201]
FolderIn = "C:\DannyData\Projects\Agrichange\Population sizes\\"
InputLocations = "LTE-FullSet"
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(2,len(Years)):
        InFile = FolderIn + "ppp_" + str(Years[year]) + "_1km_Aggregated.tif" 
        LTECount = 0;
        Start_Seq = 0
        expression=" Idnr >= "+ str(Start_Seq)
        with arcpy.da.SearchCursor(InputLocations, ["Locations"], where_clause=expression) as cursor: 
            for row in cursor: 
                Message = "Running LTE: " + (row[0]) + " for " + str(Years[year]) + "           "
                print(Message, end="\r")
                exp = "Locations = '" + (row[0]) + "'"
                LTECount = LTECount + 1;

                arcpy.analysis.Select(in_features=InputLocations,out_feature_class="TestedLTE", where_clause=exp)
                arcpy.management.Project(in_dataset="TestedLTE",out_dataset="LTE_Project",
                        out_coor_system='PROJCS["WGS_1984_World_Mercator",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]',
                        transform_method=None,in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
                        preserve_shape="NO_PRESERVE_SHAPE",max_deviation=None,vertical="NO_VERTICAL")
                arcpy.analysis.Buffer(in_features="LTE_Project", out_feature_class="LTE_Buffer",buffer_distance_or_field="25000 Meters",line_side="FULL",
                                     line_end_type="ROUND",dissolve_option="NONE",dissolve_field=None,method="PLANAR") 
                ClipextentBuffered = arcpy.Describe("LTE_Buffer").extent
                Extent = str(ClipextentBuffered)   
               
               # This is the buffer clipped population layer
                arcpy.management.Clip(in_raster=InFile,rectangle=Extent,out_raster="InClipped",in_template_dataset="LTE_Buffer",
                     nodata_value="-999",clipping_geometry="ClippingGeometry",maintain_clipping_extent="MAINTAIN_EXTENT")
                arcpy.Delete_management("ProjectInRast") 
                arcpy.management.ProjectRaster(in_raster="InClipped",out_raster="ProjectInRast",
                       out_coor_system='PROJCS["WGS_1984_World_Mercator",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]',
                       resampling_type="NEAREST",cell_size="1000 1000",geographic_transform=None,Registration_Point=None,
                       in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
                       vertical="NO_VERTICAL")

                # Distance raster
                arcpy.Delete_management("WeightFraction") 
                arcpy.Delete_management("DistanceRast") 
                arcpy.Delete_management("KMRast") 
                arcpy.Delete_management("MinusfactorRast") 
                arcpy.Delete_management("ExpRastpreD2") 
                arcpy.Delete_management("Two_D_Pi_correction") 
                arcpy.Delete_management("ConRast") 
                out_raster = arcpy.sa.Con(in_conditional_raster="ProjectInRast",in_true_raster_or_constant=1,in_false_raster_or_constant=0,
                                          where_clause="VALUE >= -1")
                out_raster.save("ConRast")
                out_distance_raster = arcpy.sa.CostDistance(in_source_data="LTE_Project",in_cost_raster="ConRast",maximum_distance=None,
                       out_backlink_raster=None,source_cost_multiplier=None,source_start_cost=None,source_resistance_rate=None,
                       source_capacity=None,source_direction="")
                out_distance_raster.save("DistanceRast")
                out_raster = arcpy.sa.Divide(in_raster_or_constant1="DistanceRast",in_raster_or_constant2=1000)
                out_raster.save("KMRast")
                out_raster = arcpy.sa.Minus(in_raster_or_constant1=0,in_raster_or_constant2="KMRast")
                out_raster.save("MinusfactorRast")
                out_raster = arcpy.sa.Exp(in_raster_or_constant="MinusfactorRast")
                out_raster.save("ExpRastpreD2") 
                out_raster = arcpy.sa.Times(in_raster_or_constant1="KMRast",in_raster_or_constant2=6.28318530717959)
                out_raster.save("Two_D_Pi_correction")
                # This is the distance raster after applying a exponential curve and 2D correction that will be used as weighting on population
                out_raster = arcpy.sa.Divide(in_raster_or_constant1="ExpRastpreD2",in_raster_or_constant2="Two_D_Pi_correction")
                out_raster.save("WeightFraction") 
               
                arcpy.Delete_management("Deposition") 
                # the actual weighting 
                out_raster = arcpy.sa.Times(in_raster_or_constant1="WeightFraction",in_raster_or_constant2="ProjectInRast")
                out_raster.save("Deposition")
                
               # The extraction 
                TableLTEYear = "LTEYear"
                if LTECount == 1:
                    LTEtmpn = TableLTEYear
                else:
                    LTEtmpn = "TableTmp"
                NewName = "LTE_" + row[0]
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data="LTE_Buffer",zone_field="OBJECTID",in_value_raster="Deposition",
                     out_table=LTEtmpn,ignore_nodata="DATA",statistics_type="SUM",process_as_multidimensional="CURRENT_SLICE",
                     percentile_values=[90],percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                arcpy.management.AlterField(in_table=LTEtmpn,field="SUM",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
                if LTECount != 1:
                    arcpy.management.JoinField(in_data=TableLTEYear,in_field="OBJECTID",join_table=LTEtmpn,join_field="OBJECTID",
                        fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)    
           
            arcpy.management.DeleteField(in_table=TableLTEYear,drop_field="Locations;ZONE_CODE;AREA;OBJECTID_1",method="DELETE_FIELDS")
            arcpy.management.AlterField(in_table=TableLTEYear,field="COUNT",new_field_name="Year",new_field_alias="YEAR",
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            arcpy.management.CalculateField(in_table=TableLTEYear,field="Year",expression=Years[year],expression_type="PYTHON3",
                                        code_block="",field_type="DOUBLE",enforce_domains="NO_ENFORCE_DOMAINS")
        
            #end of per LTE loop 
            OutFile =  "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\PopulationPressure_" + str(Years[year]) + ".csv"
            fields = arcpy.ListFields(TableLTEYear)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(TableLTEYear):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals) 
print("Finished                              ")                            

Finished                              
